<a href="https://colab.research.google.com/github/Ewallk/Projects/blob/main/Impactos_em_navios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fazendo as importações necessárias**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime as dt
from datetime import timedelta

#**Gerando Dataframe com a tabela de impactos**

In [ ]:
df = pd.read_excel('Tempos Inspetoria_fev22.xlsx')
df



,Código Porto,Código Nome Navio,Código Píer,Número Embarque,Descrição Grupo Evento,Sigla Evento Situação,Descrição Evento Situação,Data Hora Evento Início,Data Hora Evento Fim,Duração Evento,Mês (Data Hora Evento Início),Dia (Data Hora Evento Início),ACL < 4 min,Ok Oficial > 8 min
0,PM,CATS - CAPE TSUBAKI,4N,13907,Manobra de Atracação / Desatracação,MDT,Manobra de desatracacao,2022-01-01 00:10:00.000,2022-01-01 00:18:00.000,0.133333,1,1,NaN,NaN
1,PM,ORDO - ORE DONGJIAKOU,-2,14072,Manobra de Atracação / Desatracação,MAN,Manobra de Atracacao.,2022-01-01 03:39:59.999,2022-01-01 05:42:00.000,2.033333,1,1,NaN,NaN
2,PM,ORDO - ORE DONGJIAKOU,-2,14072,Amarração,IAM,Inicio de Amarracao.,2022-01-01 05:42:00.000,2022-01-01 06:19:00.000,0.616667,1,1,NaN,NaN
3,PM,ORDO - ORE DONGJIAKOU,4N,14072,Aguardando Arqueação,AIN,Arqueacao Inicial.,2022-01-01 06:19:00.000,2022-01-01 06:19:59.999,0.016667,1,1,NaN,NaN
4,PM,HLAQ - HL AQUAMARINE,3S,14012,Parada Trimming/Outros,ACL,Aguardando Checar Calados,2022-01-02 06:12:59.999,2022-01-02 06:17:00.000,0.066667,1,2,Atenção!,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,PM,CPTI - CP TIANJIN,3N,10424,Manobra de Atracação / Desatracação,MDT,Manobra de desatracacao,2022-01-31 00:17:00.000,2022-01-31 00:26:00.000,0.150000,1,31,NaN,NaN
493,PM,YUU2 - YUAN FU HAI,4S,14073,Aguardando Arqueação,AIN,Arqueacao Inicial.,2022-01-31 02:49:00.000,2022-01-31 02:50:00.000,0.016667,1,31,NaN,NaN
494,PM,YUSU - YUAN SUI HAI,4N,13726,Parada Trimming/Outros,PPT,Paralisacao para Trimming,2022-01-31 20:22:00.000,2022-01-31 20:33:00.000,0.183333,1,31,NaN,NaN
495,PM,YUSU - YUAN SUI HAI,4N,13726,Operando,TRI,Trimming,2022-01-31 20:52:59.999,2022-01-31 21:41:59.999,0.816667,1,31,NaN,NaN


#**Verificando o tipo dos dados**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Código Porto                   497 non-null    object        
 1   Código Nome Navio              497 non-null    object        
 2   Código Píer                    497 non-null    object        
 3   Número Embarque                497 non-null    int64         
 4   Descrição Grupo Evento         497 non-null    object        
 5   Sigla Evento Situação          497 non-null    object        
 6   Descrição Evento Situação      497 non-null    object        
 7   Data Hora Evento Início        497 non-null    datetime64[ns]
 8   Data Hora Evento Fim           497 non-null    datetime64[ns]
 9   Duração Evento                 497 non-null    float64       
 10  Mês (Data Hora Evento Início)  497 non-null    int64         
 11  Dia (Data Hora Even

##Renomeando as colunas para melhor operacionalização

In [ ]:
df.rename(columns={'Código Porto':'cod_porto', 'Código Nome Navio':'nome_navio', 
                   'Código Píer':'cod_pier', 'Número Embarque': 'numero_embarque',
                   'Descrição Grupo Evento': 'desc_grupo_evento', 'Sigla Evento Situação': 'sigla_evento',
                   'Descrição Evento Situação': 'situacao_evento', 'Data Hora Evento Início': 'inicio_evento',
                   'Data Hora Evento Fim': 'fim_evento', 'Duração Evento': 'duracao_evento',
                   'Mês (Data Hora Evento Início)': 'mes_evento', 'Dia (Data Hora Evento Início)': 'dia_evento'}, inplace=True)

##Verificando as colunas renomeadas

In [ ]:
df.head()

,cod_porto,nome_navio,cod_pier,numero_embarque,desc_grupo_evento,sigla_evento,situacao_evento,inicio_evento,fim_evento,duracao_evento,mes_evento,dia_evento,ACL < 4 min,Ok Oficial > 8 min
0,PM,CATS - CAPE TSUBAKI,4N,13907,Manobra de Atracação / Desatracação,MDT,Manobra de desatracacao,2022-01-01 00:10:00.000,2022-01-01 00:18:00.000,0.133333,1,1,NaN,NaN
1,PM,ORDO - ORE DONGJIAKOU,-2,14072,Manobra de Atracação / Desatracação,MAN,Manobra de Atracacao.,2022-01-01 03:39:59.999,2022-01-01 05:42:00.000,2.033333,1,1,NaN,NaN
2,PM,ORDO - ORE DONGJIAKOU,-2,14072,Amarração,IAM,Inicio de Amarracao.,2022-01-01 05:42:00.000,2022-01-01 06:19:00.000,0.616667,1,1,NaN,NaN
3,PM,ORDO - ORE DONGJIAKOU,4N,14072,Aguardando Arqueação,AIN,Arqueacao Inicial.,2022-01-01 06:19:00.000,2022-01-01 06:19:59.999,0.016667,1,1,NaN,NaN
4,PM,HLAQ - HL AQUAMARINE,3S,14012,Parada Trimming/Outros,ACL,Aguardando Checar Calados,2022-01-02 06:12:59.999,2022-01-02 06:17:00.000,0.066667,1,2,Atenção!,NaN


Agrupar os dados por píer, é um método importante para checar as especificidades de cada berço sendo assim, vou checar em quantas categóricas está distribuída a coluna "cod_pier".

In [ ]:
df.cod_pier.unique()

array(['4N', '-2', '3S', '4S', '3N', '01'], dtype=object)

# Foi observado que alguns dados da coluna "cod_pier", apresentam o código "-2", o que não é aceitável. Por conta disso, os dados foram tratados para mostrar o código correto do píer.

In [ ]:
df.cod_pier = df.cod_pier.mask(df.cod_pier.isin(['-2'])).\
              groupby(df.numero_embarque).transform('first')

In [ ]:
df.cod_pier.unique()

array(['4N', '3S', '4S', '3N', '01'], dtype=object)

##Com a coluna "cod_pier" já tratada, vou eliminar as colunas que não são interessantes para a análise.

In [ ]:
df.drop(columns=['cod_porto','desc_grupo_evento','sigla_evento', 
                 'ACL < 4 min','Ok Oficial > 8 min', 
                 'mes_evento', 'dia_evento', 'fim_evento'], inplace=True)

In [ ]:
df

,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento
0,CATS - CAPE TSUBAKI,4N,13907,Manobra de desatracacao,2022-01-01 00:10:00.000,0.133333
1,ORDO - ORE DONGJIAKOU,4N,14072,Manobra de Atracacao.,2022-01-01 03:39:59.999,2.033333
2,ORDO - ORE DONGJIAKOU,4N,14072,Inicio de Amarracao.,2022-01-01 05:42:00.000,0.616667
3,ORDO - ORE DONGJIAKOU,4N,14072,Arqueacao Inicial.,2022-01-01 06:19:00.000,0.016667
4,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 06:12:59.999,0.066667
...,...,...,...,...,...,...
492,CPTI - CP TIANJIN,3N,10424,Manobra de desatracacao,2022-01-31 00:17:00.000,0.150000
493,YUU2 - YUAN FU HAI,4S,14073,Arqueacao Inicial.,2022-01-31 02:49:00.000,0.016667
494,YUSU - YUAN SUI HAI,4N,13726,Paralisacao para Trimming,2022-01-31 20:22:00.000,0.183333
495,YUSU - YUAN SUI HAI,4N,13726,Trimming,2022-01-31 20:52:59.999,0.816667


## Na coluna "situacao_evento", a categórica "Trimming" configura um momento de operação. Como a análise está sendo feita em cima dos impactos no carregamento, essa categórica não é interessante. Se faz necessária a eliminação de toda a entrada por index.

In [ ]:
dados_trimming = df[df['situacao_evento']=='Trimming'].index
dados_trimming

Int64Index([  7,  14,  25,  30,  43,  44,  45,  55,  57,  63,  64,  70,  83,
             85,  93,  96,  97, 110, 111, 116, 117, 118, 132, 136, 137, 143,
            144, 157, 158, 159, 160, 166, 169, 175, 176, 179, 182, 195, 196,
            206, 207, 213, 214, 218, 228, 230, 244, 254, 255, 256, 275, 276,
            277, 290, 291, 294, 309, 310, 327, 330, 334, 340, 343, 347, 362,
            369, 374, 375, 386, 387, 396, 405, 406, 410, 411, 432, 437, 438,
            446, 447, 459, 462, 476, 477, 482, 487, 489, 495],
           dtype='int64')

In [ ]:
df.drop(dados_trimming, inplace=True)
df

,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento
0,CATS - CAPE TSUBAKI,4N,13907,Manobra de desatracacao,2022-01-01 00:10:00.000,0.133333
1,ORDO - ORE DONGJIAKOU,4N,14072,Manobra de Atracacao.,2022-01-01 03:39:59.999,2.033333
2,ORDO - ORE DONGJIAKOU,4N,14072,Inicio de Amarracao.,2022-01-01 05:42:00.000,0.616667
3,ORDO - ORE DONGJIAKOU,4N,14072,Arqueacao Inicial.,2022-01-01 06:19:00.000,0.016667
4,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 06:12:59.999,0.066667
...,...,...,...,...,...,...
491,YUU2 - YUAN FU HAI,4S,14073,Manobra de Atracacao.,2022-01-30 23:47:59.999,3.016667
492,CPTI - CP TIANJIN,3N,10424,Manobra de desatracacao,2022-01-31 00:17:00.000,0.150000
493,YUU2 - YUAN FU HAI,4S,14073,Arqueacao Inicial.,2022-01-31 02:49:00.000,0.016667
494,YUSU - YUAN SUI HAI,4N,13726,Paralisacao para Trimming,2022-01-31 20:22:00.000,0.183333


In [ ]:
df.reset_index()

,index,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento
0,0,CATS - CAPE TSUBAKI,4N,13907,Manobra de desatracacao,2022-01-01 00:10:00.000,0.133333
1,1,ORDO - ORE DONGJIAKOU,4N,14072,Manobra de Atracacao.,2022-01-01 03:39:59.999,2.033333
2,2,ORDO - ORE DONGJIAKOU,4N,14072,Inicio de Amarracao.,2022-01-01 05:42:00.000,0.616667
3,3,ORDO - ORE DONGJIAKOU,4N,14072,Arqueacao Inicial.,2022-01-01 06:19:00.000,0.016667
4,4,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 06:12:59.999,0.066667
...,...,...,...,...,...,...,...
404,491,YUU2 - YUAN FU HAI,4S,14073,Manobra de Atracacao.,2022-01-30 23:47:59.999,3.016667
405,492,CPTI - CP TIANJIN,3N,10424,Manobra de desatracacao,2022-01-31 00:17:00.000,0.150000
406,493,YUU2 - YUAN FU HAI,4S,14073,Arqueacao Inicial.,2022-01-31 02:49:00.000,0.016667
407,494,YUSU - YUAN SUI HAI,4N,13726,Paralisacao para Trimming,2022-01-31 20:22:00.000,0.183333


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 409 entries, 0 to 496
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   nome_navio       409 non-null    object        
 1   cod_pier         409 non-null    object        
 2   numero_embarque  409 non-null    int64         
 3   situacao_evento  409 non-null    object        
 4   inicio_evento    409 non-null    datetime64[ns]
 5   duracao_evento   409 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 22.4+ KB


#**A partir deste ponto, inicio a inserção de colunas que serão importantes para o estudo das parlisações.**

## Inserindo a Coluna "turno"

In [ ]:
def turno(event):
    early_morning = 0
    morning = 7
    night = 19
    hour = event.hour
    if early_morning <= hour and hour < morning:
      return 'NoiteAnterior'
    elif morning <= hour and hour < night:
      return 'Dia'
    else:
      return 'Noite'
df['turno'] = df['inicio_evento'].apply(turno)

In [ ]:
df

,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento,turno
0,CATS - CAPE TSUBAKI,4N,13907,Manobra de desatracacao,2022-01-01 00:10:00.000,0.133333,NoiteAnterior
1,ORDO - ORE DONGJIAKOU,4N,14072,Manobra de Atracacao.,2022-01-01 03:39:59.999,2.033333,NoiteAnterior
2,ORDO - ORE DONGJIAKOU,4N,14072,Inicio de Amarracao.,2022-01-01 05:42:00.000,0.616667,NoiteAnterior
3,ORDO - ORE DONGJIAKOU,4N,14072,Arqueacao Inicial.,2022-01-01 06:19:00.000,0.016667,NoiteAnterior
4,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 06:12:59.999,0.066667,NoiteAnterior
...,...,...,...,...,...,...,...
491,YUU2 - YUAN FU HAI,4S,14073,Manobra de Atracacao.,2022-01-30 23:47:59.999,3.016667,Noite
492,CPTI - CP TIANJIN,3N,10424,Manobra de desatracacao,2022-01-31 00:17:00.000,0.150000,NoiteAnterior
493,YUU2 - YUAN FU HAI,4S,14073,Arqueacao Inicial.,2022-01-31 02:49:00.000,0.016667,NoiteAnterior
494,YUSU - YUAN SUI HAI,4N,13726,Paralisacao para Trimming,2022-01-31 20:22:00.000,0.183333,Noite


## Criei uma planilha para auxiliar este passo. A planilha serve de auxílio para que eu possa inserir no df, qual turma é responsável por cada paralisação.

In [ ]:
auxiliar = pd.read_excel('seta turma.xlsx')
auxiliar.head()

,data,turno,turma
0,2021-12-31,Noite,D
1,2022-01-01,NoiteAnterior,D
2,2022-01-01,Dia,A
3,2022-01-01,Noite,C
4,2022-01-01,NoiteAnterior,C


###Criando uma coluna de referência:

In [ ]:
auxiliar['data'] = auxiliar['data'].astype(str)
auxiliar['referencia'] = auxiliar['data']+auxiliar['turno']
auxiliar.head()

,data,turno,turma,referencia
0,2021-12-31,Noite,D,2021-12-31Noite
1,2022-01-01,NoiteAnterior,D,2022-01-01NoiteAnterior
2,2022-01-01,Dia,A,2022-01-01Dia
3,2022-01-01,Noite,C,2022-01-01Noite
4,2022-01-01,NoiteAnterior,C,2022-01-01NoiteAnterior


### Criando uma coluna referência no df.

In [ ]:
df['data'] = df['inicio_evento'].astype(str)
for d in df['data']:
  data, hora = d.split(' ')
  df['DATA'] = data
  df['HORA'] = hora

In [ ]:
df.drop(columns=['data', 'HORA'], inplace=True)
df.head()

,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento,turno,DATA
0,CATS - CAPE TSUBAKI,4N,13907,Manobra de desatracacao,2022-01-01 00:10:00.000,0.133333,NoiteAnterior,2022-01-31
1,ORDO - ORE DONGJIAKOU,4N,14072,Manobra de Atracacao.,2022-01-01 03:39:59.999,2.033333,NoiteAnterior,2022-01-31
2,ORDO - ORE DONGJIAKOU,4N,14072,Inicio de Amarracao.,2022-01-01 05:42:00.000,0.616667,NoiteAnterior,2022-01-31
3,ORDO - ORE DONGJIAKOU,4N,14072,Arqueacao Inicial.,2022-01-01 06:19:00.000,0.016667,NoiteAnterior,2022-01-31
4,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 06:12:59.999,0.066667,NoiteAnterior,2022-01-31


In [ ]:
df['referencia'] = df['DATA']+df['turno']
df

,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento,turno,DATA,referencia
0,CATS - CAPE TSUBAKI,4N,13907,Manobra de desatracacao,2022-01-01 00:10:00.000,0.133333,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
1,ORDO - ORE DONGJIAKOU,4N,14072,Manobra de Atracacao.,2022-01-01 03:39:59.999,2.033333,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
2,ORDO - ORE DONGJIAKOU,4N,14072,Inicio de Amarracao.,2022-01-01 05:42:00.000,0.616667,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
3,ORDO - ORE DONGJIAKOU,4N,14072,Arqueacao Inicial.,2022-01-01 06:19:00.000,0.016667,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
4,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 06:12:59.999,0.066667,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
...,...,...,...,...,...,...,...,...,...
491,YUU2 - YUAN FU HAI,4S,14073,Manobra de Atracacao.,2022-01-30 23:47:59.999,3.016667,Noite,2022-01-31,2022-01-31Noite
492,CPTI - CP TIANJIN,3N,10424,Manobra de desatracacao,2022-01-31 00:17:00.000,0.150000,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
493,YUU2 - YUAN FU HAI,4S,14073,Arqueacao Inicial.,2022-01-31 02:49:00.000,0.016667,NoiteAnterior,2022-01-31,2022-01-31NoiteAnterior
494,YUSU - YUAN SUI HAI,4N,13726,Paralisacao para Trimming,2022-01-31 20:22:00.000,0.183333,Noite,2022-01-31,2022-01-31Noite


## Aqui,eu insiro a coluna "turma" no df, com base na coluna referência e "inicio_evento".

In [ ]:
df = df.merge(auxiliar[["turma", "referencia"]], on="referencia")

In [ ]:
df

,nome_navio,cod_pier,numero_embarque,situacao_evento,inicio_evento,duracao_evento,turno,DATA,referencia,turma
0,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Checar Calados,2022-01-02 09:55:59.999,0.133333,Dia,2022-01-31,2022-01-31Dia,B
1,HLAQ - HL AQUAMARINE,3S,14012,Paralisacao para Trimming,2022-01-02 13:58:00.000,0.250000,Dia,2022-01-31,2022-01-31Dia,B
2,HLAQ - HL AQUAMARINE,3S,14012,Aguardando Ok Oficial,2022-01-02 15:07:00.000,0.016667,Dia,2022-01-31,2022-01-31Dia,B
3,HLAQ - HL AQUAMARINE,3S,14012,Manobra de desatracacao,2022-01-02 17:15:00.000,0.150000,Dia,2022-01-31,2022-01-31Dia,B
4,PASD - PASCHALIS D,3N,13751,Inicio de Amarracao.,2022-01-03 07:31:59.999,1.333333,Dia,2022-01-31,2022-01-31Dia,B
...,...,...,...,...,...,...,...,...,...,...
302,ORHI - ORE CHINA,4N,13690,Manobra de desatracacao,2022-01-28 22:31:59.999,0.350000,Noite,2022-01-31,2022-01-31Noite,D
303,CPTI - CP TIANJIN,3N,10424,Aguardando Ok Oficial,2022-01-30 19:15:00.000,0.016667,Noite,2022-01-31,2022-01-31Noite,D
304,YUU2 - YUAN FU HAI,4S,14073,Manobra de Atracacao.,2022-01-30 23:47:59.999,3.016667,Noite,2022-01-31,2022-01-31Noite,D
305,YUSU - YUAN SUI HAI,4N,13726,Paralisacao para Trimming,2022-01-31 20:22:00.000,0.183333,Noite,2022-01-31,2022-01-31Noite,D


#Exportando o df já limpo para criação do dashboard.

In [ ]:
df.to_excel('Tempos Inspetoria fev limpo.xlsx')